<a href="https://colab.research.google.com/github/map72ru/matalg/blob/main/%D0%94%D0%977.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

К алгоритму kNN, представленному на уроке, реализовать добавление весов для соседей по любому из показанных на уроке принципов.

In [ ]:
import numpy as np
from sklearn import model_selection
from sklearn.datasets import load_iris


def e_metrics(x1, x2):
    distance = 0
    for i in range(len(x1)):
        distance += np.square(x1[i] - x2[i])

    return np.sqrt(distance)


#
# фиксированный вес
#
def weight_one(distance, num, **kwargs):
    weight = 1
    if kwargs.get('weight') is not None:
        weight = kwargs.get('weight')
        if weight > 1 or weight < 0:
            raise ValueError("Weight must be in interval between 0 and 1")
    return weight


#
# Формула для вычисления веса q**i
#
def q_i(distance, num, **kwargs):
    weight = 1
    if kwargs.get('weight') is not None:
        weight = kwargs.get('weight')
        if weight > 1 or weight < 0:
            raise ValueError("Weight must be in interval between 0 and 1")
    return weight ** (num + 1)


#
# Формула для вычисления веса 1/(a+i)**b
#
def reciprocal_biased(distance, num, **kwargs):
    a = 0
    b = 1
    if kwargs.get('a') is not None:
        a = kwargs.get('a')
    if kwargs.get('b') is not None:
        b = kwargs.get('b')
    return 1 / (num + 1 + a)**b


#
# Формула для вычисления веса q**i
#
def inverse(distance, num, **kwargs):
    weight = 1
    if kwargs.get('weight') is not None:
        weight = kwargs.get('weight')
        if weight > 1 or weight < 0:
            raise ValueError("Weight must be in interval between 0 and 1")
    return weight ** (num + 1)


#
# Формула для вычисления веса (k + 1 - i) / k
#
def with_k(distance, num, **kwargs):
    if kwargs.get('weight_k') is None:
        raise ValueError("K value is missing")
    k = kwargs.get('weight_k')
    return (k + 1 - num + 1) / k


#
# Формула для вычисления веса q**d
#
def q_distance(distance, num, **kwargs):
    weight = 1
    if kwargs.get('weight') is not None:
        weight = kwargs.get('weight')
        if weight > 1 or weight < 0:
            raise ValueError("Weight must be in interval between 0 and 1")
    return weight ** (distance[0])



def knn(x_train, y_train, x_test, k, weights_func, **kwargs):
    answers = []
    for x in x_test:
        test_distances = []

        for i in range(len(x_train)):
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])

            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))

        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}

        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        i = 0
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += weights_func(d, i, **kwargs)
            i += 1

        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
    return answers


def accuracy(pred, y):
    return (sum(pred == y) / len(y))


X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=1)

k = 10

y_pred = knn(X_train, y_train, X_test, k, weight_one)

print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

y_pred = knn(X_train, y_train, X_test, k, weight_one, weight=0.5)

print(f'Точность алгоритма при k = {k} weight=0.5: {accuracy(y_pred, y_test):.3f}')

y_pred = knn(X_train, y_train, X_test, k, q_i, weight=0.5)

print(f'Точность алгоритма при k = {k} c функцией q**i weight=0.5: {accuracy(y_pred, y_test):.3f}')

y_pred = knn(X_train, y_train, X_test, k, inverse)

print(f'Точность алгоритма при k = {k} c функцией 1/i: {accuracy(y_pred, y_test):.3f}')

y_pred = knn(X_train, y_train, X_test, k, reciprocal_biased, a=2, b=2)

print(f'Точность алгоритма при k = {k} c функцией 1 / (i + a)**b: {accuracy(y_pred, y_test):.3f}')

y_pred = knn(X_train, y_train, X_test, k, with_k, weight_k=k)

print(f'Точность алгоритма при k = {k} c функцией (k + 1 - i) / k: {accuracy(y_pred, y_test):.3f}')

y_pred = knn(X_train, y_train, X_test, k, q_distance, weight=0.5)

print(f'Точность алгоритма при k = {k} c функцией q**d: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 10: 0.967
Точность алгоритма при k = 10 weight=0.5: 0.967
Точность алгоритма при k = 10 c функцией q**i weight=0.5: 1.000
Точность алгоритма при k = 10 c функцией 1/i: 0.967
Точность алгоритма при k = 10 c функцией 1 / (i + a)**b: 1.000
Точность алгоритма при k = 10 c функцией (k + 1 - i) / k: 0.967
Точность алгоритма при k = 10 c функцией q**d: 0.967
